# Movies Recommend System

### 構成

1. レビューの形態素解析
2. 特徴量行列の作成
3. 事前に作成したモデルで予測

### 準備

In [2]:
#パンダスの準備
import pandas as pd
from pandas import Series,DataFrame

In [3]:
#SQLの準備
import sqlite3
con = sqlite3.connect("movies_c.db")

In [4]:
# 関数を作って、SQL文の実行結果をDataFrameにして返します。
def sql_to_df(sql_query):

    # read_sqlの引数に、SQL文とデータベースへのConnectionを渡します。
    df = pd.read_sql(sql_query, con)

    # 結果のDataFrameを返します。
    return df

In [5]:
import MeCab
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

## 1. レビューの形態素解析

In [6]:
watch = '''
泣きたい。純粋な恋愛もので泣きたい。
最近仕事ばっかりで疲れてるから癒されたい。
'''

In [7]:
text = watch
mecab.parse('')#文字列がGCされるのを防ぐ
node = mecab.parseToNode(text)
i_id=0
while node:
    #単語を取得
    word = node.surface
    #品詞を取得
    pos = node.feature.split(",")[1]
    #print('{}'.format(word))
    #print('{0} , {1}'.format(word, pos))
        
    #次の単語に進める
    node = node.next
    
    sql_query = '''
    INSERT INTO watch_cnt_table(id, word, num) values({i_id}, "{word}", 1)
    '''.format(i_id=i_id, word=word)
        
    con.execute(sql_query)
    
i_id += 1

## 2. 特徴量行列の作成-1

In [8]:
sql_query = '''
SELECT num_words.id, watch_cnt_table.word FROM watch_cnt_table, num_words WHERE num_words.word = watch_cnt_table.word GROUP BY num_words.id, watch_cnt_table.word ORDER BY num_words.id
'''
words_df = sql_to_df(sql_query)

In [9]:
query = '''SELECT '''
for item in range(len(words_df)):
    #レビューIDごとに出現しているwordでカウントする。
    query += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{n}, '''.format(word=words_df.iloc[item, 1], n=words_df.iloc[item, 0])
    
query += '''id FROM watch_cnt_table GROUP BY id'''
#print(query)

In [10]:
want_df = sql_to_df(query)

In [11]:
want_df =  want_df.drop('id', axis=1)

In [12]:
want_df = want_df.T

## 2. 特徴量行列の作成-2

In [13]:
l = []
for num in range(0,30400):
    literal = 'a{num}'.format(num=num)
    l.append(literal)

In [14]:
all_df = pd.DataFrame(columns = l)

In [15]:
all_df = all_df.T

In [16]:
last_df = all_df.join(want_df)

In [17]:
last_df = last_df.fillna(0)

In [18]:
all_df = last_df.T

In [19]:
con.close

<function Connection.close>

## 3. 事前に作成したモデルで予測

In [20]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [21]:
from sklearn import linear_model

In [22]:
from sklearn.externals import joblib
logreg = joblib.load("model_dir/model")

In [23]:
all_numpyMatrix_data = all_df.as_matrix()
X_test = all_numpyMatrix_data
_test_data = DataFrame(X_test)

In [24]:
Y_pred = logreg.predict(X_test)

In [25]:
Y_pred

array(['イングリッシュ・ペイシェント'], dtype=object)